In [42]:
import pandas as pd
import pymysql
import numpy as np
import json
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook
import warnings
import glob
import gzip
import os
import time
warnings.filterwarnings('ignore')

In [43]:
pd.options.display.float_format = '{:,.0f}'.format

In [44]:
with open('/Users/Peach/Dropbox/Dojo-env/dojo-env-setup/Stack4/envs/tmdbenc.json', 'r') as f:
    esvar = json.load(f)

In [45]:
tmdb.API_KEY = esvar['API_KEY']

In [46]:
MovieDf = pd.DataFrame(columns=['title', 'budget', 'revenue', 'certification'])

# Custom Funcitons

In [47]:
# Function to write to json

def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)


In [48]:
#Funciton to extract certification 

def gmwr(m_id):

    movie = tmdb.Movies(m_id)
    
    info = movie.info()
    
    releases = movie.releases()
    
    for c in releases['countries']:
        if c['iso_3166_1'] == 'US':
            info['certification'] = c['certification']
    return info

In [49]:
gmwr('tt0848228')

{'adult': False,
 'backdrop_path': '/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg',
 'belongs_to_collection': {'id': 86311,
  'name': 'The Avengers Collection',
  'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg',
  'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'},
 'budget': 220000000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
  {'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'}],
 'homepage': 'https://www.marvel.com/movies/the-avengers',
 'id': 24428,
 'imdb_id': 'tt0848228',
 'original_language': 'en',
 'original_title': 'The Avengers',
 'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
 'popularity': 121.802,
 'poster_path': '/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg',
 'production_companies': [{'id': 420,
   'logo_path

In [50]:
gmwr('tt0332280')

{'adult': False,
 'backdrop_path': '/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg',
 'belongs_to_collection': None,
 'budget': 29000000,
 'genres': [{'id': 10749, 'name': 'Romance'}, {'id': 18, 'name': 'Drama'}],
 'homepage': 'http://www.newline.com/properties/notebookthe.html',
 'id': 11036,
 'imdb_id': 'tt0332280',
 'original_language': 'en',
 'original_title': 'The Notebook',
 'overview': "An epic love story centered around an older man who reads aloud to a woman with Alzheimer's. From a faded notebook, the old man's words bring to life the story about a couple who is separated by World War II, and is then passionately reunited, seven years later, after they have taken different paths.",
 'popularity': 57.441,
 'poster_path': '/rNzQyW4f8B8cQeg7Dgj3n6eT5k9.jpg',
 'production_companies': [{'id': 12,
   'logo_path': '/5ThIuO93vsk47oexKTSdfKEr7EC.png',
   'name': 'New Line Cinema',
   'origin_country': 'US'},
  {'id': 1565, 'logo_path': None, 'name': 'Avery Pix', 'origin_country': 'US'},
  {'id': 26

## Reading in "csv.gz"

In [51]:
file_path = "/Users/Peach/Dropbox/Dojo-env/dojo-env-setup/Stack4/IMDB_Project/DataMain/basics.csv.gz"

In [52]:
basics = pd.read_csv(file_path, compression='gzip')

In [53]:
basics

,Unnamed: 0,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,"2,001",NaN,118,"Comedy,Fantasy,Romance"
1,42384,tt0043139,movie,Life of a Beijing Policeman,Wo zhe yi bei zi,0,"2,013",NaN,120,"Drama,History"
2,61115,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,"2,020",NaN,70,Drama
3,67668,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,"2,018",NaN,122,Drama
4,86800,tt0088751,movie,The Naked Monster,The Naked Monster,0,"2,005",NaN,100,"Comedy,Horror,Sci-Fi"
...,...,...,...,...,...,...,...,...,...,...
81757,9926017,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,"2,019",NaN,74,Drama
81758,9926412,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,"2,019",NaN,97,"Comedy,Drama,Fantasy"
81759,9926552,tt9916170,movie,The Rehearsal,O Ensaio,0,"2,019",NaN,51,Drama
81760,9926561,tt9916190,movie,Safeguard,Safeguard,0,"2,020",NaN,95,"Action,Adventure,Thriller"


### Specifications - Financial Data
Your stakeholder would like you to extract and save the results for movies that meet all of the criteria established in part 1 of the project (You should already have a filtered dataframe saved from part one as a csv.gz file)

As a proof-of-concept, they requested you perform a test extraction of movies that started in 2000 or 2001

Each year should be saved as a separate .csv.gz file

One function will add the certification (MPGG Rating) to movie.info

The other function will help you append/extend a JSON file with Python

In [54]:
csv_files_movie = '/Users/Peach/Dropbox/Dojo-env/dojo-env-setup/Stack4/IMDB_Project/DataSecondary/'
csv_files_movie

'/Users/Peach/Dropbox/Dojo-env/dojo-env-setup/Stack4/IMDB_Project/DataSecondary/'

In [55]:
%%time

YEARS_TO_GET = [2000,2001]

errors = [ ]


existing_files = [file for file in os.listdir(csv_files_movie) if file.startswith("final")]


if len(existing_files) > 0:
    print(f'{existing_files} exist' )
else:
    
# Start of OUTER loop
    for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
        #Defining the JSON file to store results for year
        JSON_FILE = f'{csv_files_movie}tmdb_api_results_{YEAR}.json'
        # Check if file exists
        file_exists = os.path.isfile(JSON_FILE)
        # If it does not exist: create it
        if file_exists == False:
        # save an empty dict with just "imdb_id" to the new json file.
            with open(JSON_FILE,'w') as f:
                json.dump([{'imdb_id':0}],f)
        #Saving new year as the current df
        df = basics.loc[ basics['startYear']==YEAR].copy()
        # saving movie ids to list
        movie_ids = df['tconst'].copy()
        # Load existing data from json into a dataframe called "previous_df"
        previous_df = pd.read_json(JSON_FILE)
        # filter out any ids that are already in the JSON_FILEa
        movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
        # INNER Loop
        for movie_id in tqdm_notebook(movie_ids_to_get,
                                    desc=f'Movies from {YEAR}',
                                    position=1,
                                    leave=True):
            try:
                # Retrieve then data for the movie id
                temp = gmwr(movie_id)  
                # Append/extend results to existing file using a pre-made function
                write_json(temp,JSON_FILE)
                # Short 20 ms sleep to prevent overwhelming server
                time.sleep(0.02)

            except Exception as e:
                errors.append([movie_id, e])
        final_year_df = pd.read_json(JSON_FILE)
        final_year_df.to_csv(f"{csv_files_movie}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

        print(f"- Total errors: {len(errors)}")

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1448 [00:00<?, ?it/s]

- Total errors: 208


Movies from 2001:   0%|          | 0/1570 [00:00<?, ?it/s]

- Total errors: 441
CPU times: total: 3min 58s
Wall time: 17min 21s


In [56]:
errors

[['tt0115937',
  requests.exceptions.HTTPError('404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0115937?api_key=b213c9b7498043c45a0d8305b3706570')],
 ['tt0116628',
  requests.exceptions.HTTPError('404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0116628?api_key=b213c9b7498043c45a0d8305b3706570')],
 ['tt0118710',
  requests.exceptions.HTTPError('404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0118710?api_key=b213c9b7498043c45a0d8305b3706570')],
 ['tt0119966',
  requests.exceptions.HTTPError('404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0119966?api_key=b213c9b7498043c45a0d8305b3706570')],
 ['tt0120673',
  requests.exceptions.HTTPError('404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0120673?api_key=b213c9b7498043c45a0d8305b3706570')],
 ['tt0137698',
  requests.exceptions.HTTPError('404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt013

In [57]:
print(f"- Total errors: {len(errors)}")

- Total errors: 441


In [58]:
final_csv_files = [file for file in os.listdir(csv_files_movie) if os.path.basename(file).startswith("final")]
final_csv_files

['final_tmdb_data_2000.csv.gz', 'final_tmdb_data_2001.csv.gz']

In [59]:
dataframe_final = {}

for file_name in final_csv_files:
    file_path = os.path.join(csv_files_movie, file_name)
    with gzip.open(file_path, 'rt', encoding='utf-8') as file:
        df = pd.read_csv(file, index_col=None)
        file_name = os.path.splitext(os.path.basename(file_name))[0]
        dataframe_final[file_name] = df
print(dataframe_final)

{'final_tmdb_data_2000.csv':         imdb_id  adult                     backdrop_path  \
0             0    NaN                               NaN   
1     tt0113026      0  /vMFs7nw6P0bIV1jDsQpxAieAVnH.jpg   
2     tt0113092      0                               NaN   
3     tt0116391      0                               NaN   
4     tt0118694      0  /n4GJFGzsc7NinI1VeGDXIcQjtU2.jpg   
...         ...    ...                               ...   
1236  tt6174238      0                               NaN   
1237  tt7029820      0                               NaN   
1238  tt7197642      0                               NaN   
1239  tt7631368      0  /sF0gUHE0YzZNXYugTB2LFxJIppf.jpg   
1240  tt7802790      0  /etxml2M8GQzb31jcH0cdA489WCX.jpg   

                                  belongs_to_collection     budget  \
0                                                   NaN        NaN   
1                                                   NaN 10,000,000   
2                                       

In [60]:
final_2000 = dataframe_final['final_tmdb_data_2000.csv']
final_2001 = dataframe_final['final_tmdb_data_2001.csv']

In [61]:
final_2000

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0113026,0,/vMFs7nw6P0bIV1jDsQpxAieAVnH.jpg,NaN,"10,000,000","[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",NaN,"62,127",en,The Fantasticks,...,0,86,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Try to remember the first time magic happened,The Fantasticks,0,6,22,NaN
2,tt0113092,0,NaN,NaN,0,"[{'id': 878, 'name': 'Science Fiction'}]",NaN,"110,977",en,For the Cause,...,0,100,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,The ultimate showdown on a forbidden planet.,For the Cause,0,5,8,NaN
3,tt0116391,0,NaN,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",NaN,"442,869",hi,Gang,...,0,152,"[{'english_name': 'Hindi', 'iso_639_1': 'hi', ...",Released,NaN,Gang,0,4,1,NaN
4,tt0118694,0,/n4GJFGzsc7NinI1VeGDXIcQjtU2.jpg,"{'id': 1131062, 'name': 'Wong Kar-Wai’s Love T...","150,000","[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,843,cn,花樣年華,...,"14,204,632",99,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,"Feel the heat, keep the feeling burning, let t...",In the Mood for Love,0,8,"2,239",PG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1236,tt6174238,0,NaN,NaN,0,"[{'id': 80, 'name': 'Crime'}]",NaN,"223,878",cn,冷战,...,0,0,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,NaN,Cold War,0,2,2,NaN
1237,tt7029820,0,NaN,NaN,"7,000",[],NaN,"604,889",en,Scream For Christmas,...,0,80,[],Released,NaN,Scream For Christmas,0,0,0,NaN
1238,tt7197642,0,NaN,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,"872,676",en,"Goodbye, Merry-Go-Round",...,0,90,[],Released,NaN,"Goodbye, Merry-Go-Round",0,0,0,NaN
1239,tt7631368,0,/sF0gUHE0YzZNXYugTB2LFxJIppf.jpg,NaN,"10,000,000","[{'id': 27, 'name': 'Horror'}]",NaN,"97,186",fr,"I, Vampire",...,0,85,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,"I, Vampire",0,6,4,NR


In [62]:
final_2001

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0035423,0,/hfeiSfWYujh6MKhtGTXyK3DD4nN.jpg,NaN,"48,000,000","[{'id': 10749, 'name': 'Romance'}, {'id': 14, ...",NaN,"11,232",en,Kate & Leopold,...,"76,019,048",118,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,"If they lived in the same century, they'd be p...",Kate & Leopold,0,6,"1,196",PG-13
2,tt0114447,0,NaN,NaN,0,"[{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...",NaN,"151,007",en,The Silent Force,...,0,90,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,They left him for dead... They should have fin...,The Silent Force,0,5,3,NaN
3,tt0116916,0,/rFpHBidSlhjflmnLu7BZilyKeQR.jpg,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,"73,549",en,The Dark Mist,...,0,101,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,The Dark Mist,0,4,2,PG
4,tt0118589,0,/9NZAirJahVilTiDNCHLFcdkwkiy.jpg,NaN,"22,000,000","[{'id': 18, 'name': 'Drama'}, {'id': 10402, 'n...",NaN,"10,696",en,Glitter,...,"5,271,666",104,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,"In music she found her dream, her love, herself.",Glitter,0,4,126,PG-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333,tt7797790,0,NaN,NaN,0,"[{'id': 27, 'name': 'Horror'}]",NaN,"956,219",en,Edmund Kemper Part 3: La mort sévit,...,0,72,"[{'english_name': 'French', 'iso_639_1': 'fr',...",Released,NaN,Edmund Kemper Part 3: La mort sévit,0,0,0,NaN
1334,tt8665056,0,NaN,NaN,0,"[{'id': 37, 'name': 'Western'}]",http://skeletoncreekproductions.com/p-movie-br...,"885,436",en,Guns Along The Bravo,...,0,85,[],Released,Evil came to the Southwest until three blazing...,Guns Along The Bravo,0,0,0,NaN
1335,tt8795764,0,NaN,NaN,0,"[{'id': 27, 'name': 'Horror'}]",https://www.utahwolf.com/films/coming-soon-new...,"871,624",en,New Breed,...,0,57,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,New Breed,0,0,0,NR
1336,tt9071078,0,NaN,NaN,0,"[{'id': 28, 'name': 'Action'}]",NaN,"201,706",cn,致命密函,...,0,90,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,NaN,Chinese Heroes,0,3,2,NaN


In [63]:
concat_df = pd.concat(dataframe_final)

In [64]:
concat_df

imdb_id  adult  \
final_tmdb_data_2000.csv 0             0    NaN   
                         1     tt0113026      0   
                         2     tt0113092      0   
                         3     tt0116391      0   
                         4     tt0118694      0   
...                                  ...    ...   
final_tmdb_data_2001.csv 1333  tt7797790      0   
                         1334  tt8665056      0   
                         1335  tt8795764      0   
                         1336  tt9071078      0   
                         1337  tt9212730      0   

                                                  backdrop_path  \
final_tmdb_data_2000.csv 0                                  NaN   
                         1     /vMFs7nw6P0bIV1jDsQpxAieAVnH.jpg   
                         2                                  NaN   
                         3                                  NaN   
                         4     /n4GJFGzsc7NinI1VeGDXIcQjtU2.jpg   
...                                                         ...   
final_tmdb_data_2001.csv 1333                               NaN   
                         1334                               NaN   
                         1335                               NaN   
                         1336                               NaN   
                         1337                               NaN   

                                                           belongs_to_collection  \
final_tmdb_data_2000.csv 0                                                   NaN   
                         1                                                   NaN   
                         2                                                   NaN   
                         3                                                   NaN   
                         4     {'id': 1131062, 'name': 'Wong Kar-Wai’s Love T...   
...                                                                          ...   
final_tmdb_data_2001.csv 1333                                                NaN   
                         1334                                                NaN   
                         1335                                                NaN   
                         1336                                                NaN   
                         1337                                                NaN   

                                  budget  \
final_tmdb_data_2000.csv 0           NaN   
                         1    10,000,000   
                         2             0   
                         3             0   
                         4       150,000   
...                                  ...   
final_tmdb_data_2001.csv 1333          0   
                         1334          0   
                         1335          0   
                         1336          0   
                         1337          0   

                                                                          genres  \
final_tmdb_data_2000.csv 0                                                   NaN   
                         1     [{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...   
                         2              [{'id': 878, 'name': 'Science Fiction'}]   
                         3     [{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...   
                         4     [{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...   
...                                                                          ...   
final_tmdb_data_2001.csv 1333                     [{'id': 27, 'name': 'Horror'}]   
                         1334                    [{'id': 37, 'name': 'Western'}]   
                         1335                     [{'id': 27, 'name': 'Horror'}]   
                         1336                     [{'id': 28, 'name': 'Action'}]   
                         1337  [{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...   

                                                                    

In [65]:
concat_df[['budget','revenue']].isna().sum()

budget     2
revenue    2
dtype: int64

In [66]:
concat_df.dropna(subset=['budget','revenue'], inplace=True)
concat_df[['budget','revenue']].isna().sum()

budget     0
revenue    0
dtype: int64

In [67]:
concat_df

imdb_id  adult  \
final_tmdb_data_2000.csv 1     tt0113026      0   
                         2     tt0113092      0   
                         3     tt0116391      0   
                         4     tt0118694      0   
                         5     tt0118852      0   
...                                  ...    ...   
final_tmdb_data_2001.csv 1333  tt7797790      0   
                         1334  tt8665056      0   
                         1335  tt8795764      0   
                         1336  tt9071078      0   
                         1337  tt9212730      0   

                                                  backdrop_path  \
final_tmdb_data_2000.csv 1     /vMFs7nw6P0bIV1jDsQpxAieAVnH.jpg   
                         2                                  NaN   
                         3                                  NaN   
                         4     /n4GJFGzsc7NinI1VeGDXIcQjtU2.jpg   
                         5     /vceiGZ3uavAEHlTA7v0GjQsGVKe.jpg   
...                                                         ...   
final_tmdb_data_2001.csv 1333                               NaN   
                         1334                               NaN   
                         1335                               NaN   
                         1336                               NaN   
                         1337                               NaN   

                                                           belongs_to_collection  \
final_tmdb_data_2000.csv 1                                                   NaN   
                         2                                                   NaN   
                         3                                                   NaN   
                         4     {'id': 1131062, 'name': 'Wong Kar-Wai’s Love T...   
                         5                                                   NaN   
...                                                                          ...   
final_tmdb_data_2001.csv 1333                                                NaN   
                         1334                                                NaN   
                         1335                                                NaN   
                         1336                                                NaN   
                         1337                                                NaN   

                                  budget  \
final_tmdb_data_2000.csv 1    10,000,000   
                         2             0   
                         3             0   
                         4       150,000   
                         5             0   
...                                  ...   
final_tmdb_data_2001.csv 1333          0   
                         1334          0   
                         1335          0   
                         1336          0   
                         1337          0   

                                                                          genres  \
final_tmdb_data_2000.csv 1     [{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...   
                         2              [{'id': 878, 'name': 'Science Fiction'}]   
                         3     [{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...   
                         4     [{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...   
                         5                         [{'id': 18, 'name': 'Drama'}]   
...                                                                          ...   
final_tmdb_data_2001.csv 1333                     [{'id': 27, 'name': 'Horror'}]   
                         1334                    [{'id': 37, 'name': 'Western'}]   
                         1335                     [{'id': 27, 'name': 'Horror'}]   
                         1336                     [{'id': 28, 'name': 'Action'}]   
                         1337  [{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...   

                                                                    

In [68]:
concat_df[['budget','revenue']] = concat_df[['budget','revenue']].astype('float64')

# Dataset publication

In [73]:
output_file = "/Users/Peach/Dropbox/Dojo-env/dojo-env-setup/Stack4/IMDB_Project/Datasets/tmdb_results_combined.csv.gz"

In [74]:
concat_df.to_csv(output_file, compression='gzip', index=False)